In [60]:
import requests
import pandas as pd
import sqlalchemy as sa
import pyodbc
from datetime import datetime
import pytz

## Current Weather Data

In [61]:
locations = [
    {'Province': 'Ontario', 'City': 'North York', 'PostalCode': 'M3J1P3', 'Latitude': '43.773784', 'Longitude': '-79.500362'},
    {'Province': 'Manitoba', 'City': 'Winnipeg', 'PostalCode': 'R0G2W0', 'Latitude': '49.798866', 'Longitude': '-97.064934'},
    {'Province': 'Quebec', 'City': 'Saguenay-Lac-Saint-Jean', 'PostalCode': 'G0V0A1', 'Latitude': '48.211288', 'Longitude': '-70.069969'},
    {'Province': 'Saskatchewan', 'City': 'Regina', 'PostalCode': 'S0A0A0', 'Latitude': '50.615061', 'Longitude': '-103.412753'},
    {'Province': 'Ontario', 'City': 'Thunder Bay', 'PostalCode': 'P0M2J0', 'Latitude': '48.68475', 'Longitude': '-85.641417'},
    {'Province': 'Saskatchewan', 'City': 'Saskatoon', 'PostalCode': 'S0G0A6', 'Latitude': '51.627971', 'Longitude': '-106.439486'},
    {'Province': 'Ontario', 'City': 'Sudbury', 'PostalCode': 'P0H0B9', 'Latitude': '46.442016', 'Longitude': '-80.311475'},
    {'Province': 'Alberta', 'City': 'Edmonton', 'PostalCode': 'T0A0A7', 'Latitude': '53.825792', 'Longitude': '-113.326611'},
    {'Province': 'Nunavut', 'City': 'Baffin', 'PostalCode': 'X0A0A0', 'Latitude': '73.005278', 'Longitude': '-85.033056'},
    {'Province': 'Alberta', 'City': 'Edmonton', 'PostalCode': 'T0A0A9', 'Latitude': '53.83726', 'Longitude': '-113.412126'},
    {'Province': 'Quebec', 'City': 'Gaspésie-Îles-de-la-Madeleine', 'PostalCode': 'G0C2Y0', 'Latitude': '48.175265', 'Longitude': '-65.249235'},
    {'Province': 'Nunavut', 'City': 'Keewatin', 'PostalCode': 'X0C0H0', 'Latitude': '66.556724', 'Longitude': '-86.21719'},
    {'Province': 'Northwest Territories', 'City': 'Region 6', 'PostalCode': 'X1A1Y1', 'Latitude': '62.448206', 'Longitude': '-114.373354'}
]

unit = 'metric'
current_key = 'bc69220a7f832721d51484c18d70f9bf'
headers = {"accept": "application/json", 'user-agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"} 

current_weather_data = []

for location in locations:
    lat = location['Latitude']
    lon = location['Longitude']
    pro = location['Province']
    city = location['City']
    code = location['PostalCode']
    current_url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={current_key}&units={unit}'
    
    response = requests.get(current_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        current_time = datetime.utcfromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S')
        weather_info = {
            'Latitude': lat,
            'Longitude': lon,
            'Country': data['sys']['country'],
            'Province': pro,
            'City': city,
            'Postal Code': code,
            'Current Time(UTC)': current_time,
            'Current Temp': data['main']['temp'],
            'Min Temp': data['main']['temp_min'],
            'Max Temp': data['main']['temp_max'],
            'Weather': data['weather'][0]['description'],
            'Wind Speed': data['wind']['speed'],
            'Humidity': data['main']['humidity']
        }
        current_weather_data.append(weather_info)
    else:
        print(f"Failed to fetch data for location ({lat}, {lon})")

current_weather_data= pd.DataFrame(current_weather_data)

print(current_weather_data)

/var/folders/ft/gq7wv6zs1slgbv35cnp736j00000gp/T/ipykernel_2450/3281716888.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  current_time = datetime.utcfromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S')


     Latitude    Longitude Country               Province  \
0   43.773784   -79.500362      CA                Ontario   
1   49.798866   -97.064934      CA               Manitoba   
2   48.211288   -70.069969      CA                 Quebec   
3   50.615061  -103.412753      CA           Saskatchewan   
4    48.68475   -85.641417      CA                Ontario   
5   51.627971  -106.439486      CA           Saskatchewan   
6   46.442016   -80.311475      CA                Ontario   
7   53.825792  -113.326611      CA                Alberta   
8   73.005278   -85.033056      CA                Nunavut   
9    53.83726  -113.412126      CA                Alberta   
10  48.175265   -65.249235      CA                 Quebec   
11  66.556724    -86.21719      CA                Nunavut   
12  62.448206  -114.373354      CA  Northwest Territories   

                             City Postal Code    Current Time(UTC)  \
0                      North York      M3J1P3  2024-08-02 21:56:50   
1    

## Feaure Engineering

In [62]:
# Temperature Difference
current_weather_data['Temperature_Range'] = current_weather_data['Max Temp'] - current_weather_data['Min Temp']


In [63]:

# Extract day, month, and hour from Current Time
current_weather_data['Current Time(UTC)'] = pd.to_datetime(current_weather_data['Current Time(UTC)'])


In [64]:
current_weather_data['Day'] = current_weather_data['Current Time(UTC)'].dt.day
current_weather_data['Month'] = current_weather_data['Current Time(UTC)'].dt.month
current_weather_data['Hour'] = current_weather_data['Current Time(UTC)'].dt.hour

In [65]:
current_weather_data

,Latitude,Longitude,Country,Province,City,Postal Code,Current Time(UTC),Current Temp,Min Temp,Max Temp,Weather,Wind Speed,Humidity,Temperature_Range,Day,Month,Hour
0,43.773784,-79.500362,CA,Ontario,North York,M3J1P3,2024-08-02 21:56:50,26.75,25.41,28.39,broken clouds,1.54,65,2.98,2,8,21
1,49.798866,-97.064934,CA,Manitoba,Winnipeg,R0G2W0,2024-08-02 21:56:50,28.25,26.68,28.93,broken clouds,3.60,46,2.25,2,8,21
2,48.211288,-70.069969,CA,Quebec,Saguenay-Lac-Saint-Jean,G0V0A1,2024-08-02 21:56:50,25.05,15.36,25.05,overcast clouds,2.06,73,9.69,2,8,21
3,50.615061,-103.412753,CA,Saskatchewan,Regina,S0A0A0,2024-08-02 21:56:50,32.81,32.81,32.81,broken clouds,5.45,18,0.00,2,8,21
4,48.68475,-85.641417,CA,Ontario,Thunder Bay,P0M2J0,2024-08-02 21:56:50,24.16,24.16,24.16,clear sky,3.92,31,0.00,2,8,21
5,51.627971,-106.439486,CA,Saskatchewan,Saskatoon,S0G0A6,2024-08-02 21:56:50,32.22,32.22,32.22,scattered clouds,7.06,26,0.00,2,8,21
6,46.442016,-80.311475,CA,Ontario,Sudbury,P0H0B9,2024-08-02 21:56:50,32.89,31.49,32.89,overcast clouds,3.32,36,1.40,2,8,21
7,53.825792,-113.326611,CA,Alberta,Edmonton,T0A0A7,2024-08-02 21:56:51,25.08,23.96,26.84,few clouds,2.06,35,2.88,2,8,21
8,73.005278,-85.033056,CA,Nunavut,Baffin,X0A0A0,2024-08-02 21:56:51,5.86,5.86,5.86,light rain,4.12,75,0.00,2,8,21
9,53.83726,-113.412126,CA,Alberta,Edmonton,T0A0A9,2024-08-02 21:56:51,24.71,23.68,26.56,few clouds,3.51,27,2.88,2,8,21


## Pushing to Datagrip

In [68]:
# Database Connection
connection_url = sa.engine.URL.create(
    drivername = "mssql+pyodbc",
    username   = "aimerliu",
    password   = "2024!Schulich",
    host       = "mban2024-ms-sql-server.c1oick8a8ywa.ca-central-1.rds.amazonaws.com",
    port       = "1433",
    database   = "aimerliu_db",
    query = {
        "driver" : "ODBC Driver 18 for SQL Server",
        "TrustServerCertificate" : "yes"
    }
)
my_engine = sa.create_engine(connection_url)

In [69]:
# Ingest the data into your own database in our Microsoft SQL Server
current_weather_data.to_sql(
    name='Final Current Weather Data',
    con=my_engine,
    schema = 'uploads',
    if_exists='replace',
    index=False,
    dtype= {
        'Latitude': sa.types.String,
        'Longitude': sa.types.String,
        'Country': sa.types.VARCHAR(10),
        'Province': sa.types.VARCHAR(30),
        'City': sa.types.VARCHAR(40),
        'Postal Code': sa.types.VARCHAR(10),
        'Current Time(UTC)': sa.types.DATETIME,
        'Current Temp': sa.types.FLOAT,
        'Max Temp': sa.types.FLOAT,
        'Min Temp': sa.types.FLOAT,
        'Weather': sa.types.VARCHAR(30),
        'Wind Speed':sa.types.FLOAT,
        'Humidity': sa.types.FLOAT,
        'Temperature_Range': sa.types.DECIMAL(10,3),
        'Day': sa.types.Integer,
        'Month': sa.types.Integer,
        'Hour': sa.types.Integer,
    },
    method='multi'
)

13